In [7]:
import glob
import argparse
import os
from transformers import AutoTokenizer, AutoModel #pip install transformers
from sklearn.linear_model import LogisticRegression #pip install sklearn
from sklearn import svm
import random
import numpy as np
data_dir = ".\papers_processed\*"
chunk_size = 750
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")
mathy_embeddings = [[], []]
non_mathy_embeddings = [[], []]
data = [mathy_embeddings, non_mathy_embeddings]
mathy_test = []
mathy_train = []
non_mathy_test = []
non_mathy_train = []

In [11]:
for j, dir_path in enumerate(glob.glob(data_dir)): #mathy, non_mathy
    dir_path = os.path.join(dir_path, "*")
    for k, dirr in enumerate(glob.glob(dir_path)): #test, train
        dirr = os.path.join(dirr, "*")
        for l, pdf in enumerate(glob.glob(dirr)): #txts
            pdf = open(pdf, "r", encoding="utf8")
            pdf_text = pdf.read()
            pdf.close()
            embedding_vector = None
            r = range(0, len(pdf_text), chunk_size)
            n = len(r)
            for i in r:#txt-content
                inputs = tokenizer(pdf_text[i:i+chunk_size], return_tensors="pt")
                outputs = model(**inputs)
                if embedding_vector is None:
                    embedding_vector = outputs.last_hidden_state[0][0]
                else:
                    embedding_vector += outputs.last_hidden_state[0][0]
            embedding_vector /= n
            # embedding_vector.requires_grad=False
            data[j][k].append([embedding_vector.detach().numpy(), j])
mathy_test = data[0][0]
mathy_train = data[0][1]
non_mathy_test = data[1][0]
non_mathy_train = data[1][1]
training_data = mathy_train + non_mathy_train
random.shuffle(training_data)
train_x, train_y = zip(*training_data)
test_data = mathy_test + non_mathy_test
random.shuffle(test_data)
test_x, test_y = zip(*test_data)
print("data processed")

TypeError: 'BertTokenizer' object is not callable

In [ ]:
print("LR:\n")
# x, y = self.get_training_data()
x, y = train_x, train_y
model = LogisticRegression(max_iter=300).fit(x, y)
score = model.score(x, y)
predictions = model.predict(x)
print(predictions)
print(y)
print(score)

In [ ]:
x, y = train_x, train_y
predictions = self.lr.predict(x)
mathy_predictions = predictions==0
non_mathy_predictions = predictions
total_mathy_predictions = np.sum(mathy_predictions)
total_non_mathy_predictions = np.sum(non_mathy_predictions)
mathy_template = y==0
correct_mathy = mathy_predictions